In [ ]:
import numpy as np

In [ ]:
class Boite():

  def __init__(self):
    pass


  def forward(self, inputs):

    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):
    assert self.output.shape == derivee_output.shape, f"Output shape : {self.output.shape} diff from Derivee output: {derivee_output.shape}"
    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f"Input shape : {self.inputs.shape} diff from Derivee input: {self.derivee_inputs.shape}"
    return self.derivee_inputs


  def operation(self):
    raise NotImplementedError

  def gradient(self, derivee_output):
    raise NotImplementedError


In [ ]:
class BoiteParam():

  def __init__(self, param):
    self.param = param


  def forward(self, inputs):

    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):
    assert self.output.shape == derivee_output.shape, f"Output shape : {self.output.shape} diff from Derivee output: {derivee_output.shape}"

    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f"Input shape : {self.inputs.shape} diff from Derivee input: {self.derivee_inputs.shape}"

    self.derivee_param = self.gradient_param(derivee_output)
    assert self.derivee_param.shape == self.param.shape, f"Param shape : {self.inputs.shape} diff from Derivee Param: {self.derivee_param.shape}"

    return self.derivee_inputs


  def operation(self):
    raise NotImplementedError

  def gradient(self, derivee_output):
    raise NotImplementedError

  def gradient_param(self, derivee_output):
    raise NotImplementedError

In [ ]:
class Dot(BoiteParam):

  def __init__(self, weights):
    super().__init__(weights)

  def operation(self):
    return np.dot(self.inputs, self.param)

  def gradient(self, derivee_output):
    return np.dot( derivee_output, self.param.T)

  def gradient_param(self, derivee_output):
    return np.dot(self.inputs.T, derivee_output)

  def __repr__(self):
    return "DotProduct"


In [ ]:
X = np.array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [ ]:
X.shape

(4, 3)

In [ ]:
W = np.array([[ 0.49671415],
 [-0.1382643 ],
 [ 0.64768854]])

In [ ]:
W.shape

(3, 1)

In [ ]:
M = Dot(weights=W)

In [ ]:
print(M)

DotProduct


In [ ]:
out = M.forward(X) #  (4,1)

In [ ]:
out

array([[-0.71674168],
       [ 0.64784656],
       [-0.81703373],
       [ 1.89943266]])

In [ ]:
np.dot(X, W)

array([[-0.71674168],
       [ 0.64784656],
       [-0.81703373],
       [ 1.89943266]])

In [ ]:
d_out = np.random.rand(4, 1)
d_out

array([[0.42575139],
       [0.80114159],
       [0.87407416],
       [0.06200555]])

In [ ]:
M.backward(d_out)

array([[ 0.21147674, -0.05886622,  0.2757543 ],
       [ 0.39793836, -0.11076928,  0.51889023],
       [ 0.434165  , -0.12085325,  0.56612782],
       [ 0.03079904, -0.00857315,  0.04016029]])

In [ ]:
M.derivee_param

array([[-0.53062157],
       [ 4.62332146],
       [ 1.60187136]])

In [ ]:
M.param

array([[ 0.49671415],
       [-0.1382643 ],
       [ 0.64768854]])

In [ ]:
class Add(BoiteParam):

  def __init__(self, biases):
    super().__init__(biases)

  def operation(self):
    return self.inputs + self.param

  def gradient(self, derivee_output):
    return np.ones_like(self.inputs) * derivee_output

  def gradient_param(self, derivee_output):
    r =  np.ones_like(self.param) * derivee_output
    r = np.sum(r, axis=0).reshape(1, self.param.shape[1])
    return r

  def __repr__(self):
    return "AddBias"



In [ ]:
B =np.array([[1.52302986]])
B.shape

(1, 1)

In [ ]:
b = Add(biases=B)

In [ ]:
print(b)

Add Bias


In [ ]:
out

array([[-0.71674168],
       [ 0.64784656],
       [-0.81703373],
       [ 1.89943266]])

In [ ]:
out + B

array([[0.80628818],
       [2.17087642],
       [0.70599613],
       [3.42246252]])

In [ ]:
b.forward(out)

array([[0.80628818],
       [2.17087642],
       [0.70599613],
       [3.42246252]])

In [ ]:
b.output

array([[0.80628818],
       [2.17087642],
       [0.70599613],
       [3.42246252]])

In [ ]:
d_out = np.random.randn(4, 1)

In [ ]:
b.backward(d_out)

array([[0.82435996],
       [1.16301281],
       [1.63913512],
       [0.25772603]])

In [ ]:
b.derivee_inputs

array([[0.82435996],
       [1.16301281],
       [1.63913512],
       [0.25772603]])

In [ ]:
b.derivee_param

array([[3.88423392]])

In [ ]:
class Sigmoid(Boite):

  def __init__(self):
    super().__init__()

  def operation(self):
    return 1 / (1 + np.exp(-1 * self.inputs))

  def gradient(self, derivee_output):
    return self.output * (1 - self.output) *  derivee_output

In [ ]:
sig = Sigmoid()

In [ ]:
d = np.array([1, 5, 7])

In [ ]:
d.shape

(3,)

In [ ]:
sig.forward(d)

array([0.73105858, 0.99330715, 0.99908895])

In [ ]:
sig.output

array([0.73105858, 0.99330715, 0.99908895])

In [ ]:
d_out = np.random.randn(3)

In [ ]:
sig.backward(d_out)

array([ 0.0995393 , -0.00412492,  0.00138226])

In [ ]:
sig.derivee_inputs

array([ 0.0995393 , -0.00412492,  0.00138226])

In [ ]:
class Loss():

  "La fonction perte"

  def __init__(self):
    pass

  def forward(self, prediction, target):

    assert prediction.shape == target.shape, f"Prediction shape : {prediction.shape} diff from Target shape: {target.shape}"

    self.prediction = prediction
    self.target = target

    loss = np.mean(  (self.target - self.prediction)** 2)

    return loss

  def backward(self):

    self.loss_derivee = -2 * (self.target - self.prediction) / self.prediction.shape[0]

    assert self.loss_derivee.shape == self.prediction.shape, f"Loss derivee shape : {self.loss_derivee.shape} diff from Prediction shape: {self.prediction.shape}"
    return self.loss_derivee

In [ ]:
Y = np.random.randn(4, 1)
Y

array([[ 0.97586535],
       [-2.34996566],
       [-0.27417641],
       [-1.16578877]])

In [ ]:
P = np.random.randn(4, 1)
P

array([[-0.62571636],
       [ 0.27082904],
       [ 0.83458505],
       [ 0.0296227 ]])

In [ ]:
mse = Loss()
mse.forward(P, Y)

3.0229973555775005

In [ ]:
mse.backward()

array([[-0.80079085],
       [ 1.31039735],
       [ 0.55438073],
       [ 0.59770574]])

# Dense

In [ ]:
# Dense(neurones=5) [Dot(weights), Add(biaises), Sigmoid()]

In [307]:
class Dense():


  def __init__(self, neurones, activation=None):
    self.neurones = neurones
    self.activation = activation
    self.params = []
    self.graph = []
    self.first_pass = True


  def build(self, inputs):

    # weights initialisation

    np.random.seed(42)

    self.weights = np.random.randn(inputs.shape[1], self.neurones)
    self.params.append(self.weights)

    self.biases = np.random.randn(1, self.neurones)
    self.params.append(self.biases)

    # construction du graphe

    self.graph = [Dot(weights=self.params[0]), Add(biases=self.params[1])]
    if self.activation:
      self.graph.append(self.activation)


  def forward(self, inputs):
    if self.first_pass:
      self.build(inputs)
      self.first_pass = False

    for boite in self.graph:
      inputs = boite.forward(inputs)


    self.output = inputs
    return self.output


  def backward(self, derivee_output):

    assert self.output.shape == derivee_output.shape, f"Output shape : {self.output.shape} diff from Derivee output: {derivee_output.shape}"

    for boite in reversed(self.graph):
      derivee_output = boite.backward(derivee_output)

    derivee_inputs = derivee_output

    self.get_params_gradients() # self.derivee_params



    return derivee_inputs


  def get_params_gradients(self):
    self.derivee_params = []

    for boite in self.graph:
      if issubclass(boite.__class__, BoiteParam):
        self.derivee_params.append(boite.derivee_param)



  def __repr__(self):
    r =  f"DenseLayer(neurones={self.neurones})"
    if self.activation:
      r = f"DenseLayer(neurones={self.neurones}) with Sigmoid"
    return r

In [ ]:
couche = Dense(neurones=1)

In [ ]:
print(couche)

DenseLayer(neurones=1)


In [308]:
X

array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [ ]:
couche.forward(X)

array([[0.80628818],
       [2.17087642],
       [0.7059961 ],
       [3.4224625 ]])

In [ ]:
couche.params

[array([[ 0.49671415],
        [-0.1382643 ],
        [ 0.64768854]]),
 array([[1.52302986]])]

In [ ]:
couche.params[1].shape

(1, 1)

In [ ]:
m = Dot(weights=couche.params[0])
b = Add(biases=couche.params[1])

In [ ]:
out = m.forward(X)
out

array([[-0.71674167],
       [ 0.64784657],
       [-0.81703375],
       [ 1.89943265]])

In [ ]:
b.forward(out)

array([[0.80628818],
       [2.17087642],
       [0.7059961 ],
       [3.4224625 ]])

In [ ]:
couche.graph

[DotProduct, AddBias]

In [ ]:
d_out = np.random.randn(4, 1)

In [ ]:
couche.backward(d_out)

array([[-0.1163073 ,  0.03237505, -0.15165846],
       [-0.11629914,  0.03237278, -0.15164782],
       [ 0.78441736, -0.21834876,  1.02283804],
       [ 0.38119569, -0.10610883,  0.49705868]])

In [ ]:
b.backward(d_out)

array([[-0.23415337],
       [-0.23413696],
       [ 1.57921282],
       [ 0.76743473]])

In [ ]:
m.backward(b.backward(d_out))

array([[-0.1163073 ,  0.03237505, -0.15165846],
       [-0.11629914,  0.03237278, -0.15164782],
       [ 0.78441736, -0.21834876,  1.02283804],
       [ 0.38119569, -0.10610883,  0.49705868]])

# Model

In [ ]:
# Model([Dense(neurones=3), Dense(neurones=1)])

In [317]:
class Model():

  def __init__(self, layers):

    self.layers = layers
    self.compiled = False


  def forward(self, inputs):
    for layer in self.layers:
      inputs = layer.forward(inputs)
    output = inputs

    # self.get_params()
    return output


  def backward(self, loss_derivee):


    for layer in reversed(self.layers):
      loss_derivee = layer.backward(loss_derivee)
      #print(layer)
      #print(loss_derivee)

    # self.get_params_gradients()

    return None # ce n'est pas reçu par quelqu'un


  # def get_params(self):
  #   self.params = []
  #   for layer in self.layers:
  #     self.params.append(layer.params)


  # def get_params_gradients(self):
  #   self.derivee_params = []
  #   for layer in self.layers:
  #     self.derivee_params.append(layer.derivee_params)


  def parameters(self):
    for layer in self.layers:
      yield from layer.params

  def gradients(self):
    for layer in self.layers:
      yield from layer.derivee_params


  def step(self): # optimizer
    for (param, derivee_param) in zip(self.parameters(), self.gradients() ):
      param -= self.learning_rate * derivee_param

  def update(self): # optimizer
    for (param, derivee_param) in zip(self.parameters(), self.gradients() ):
      param -= 0.01 * derivee_param


  def compile(self, loss, learning_rate):
    self.loss = loss
    self.learning_rate = learning_rate

    for layer in self.layers:
      layer.first_pass=True # build the layer again

    self.compiled = True



  def fit(self, X, y, epochs=1):
    self.history = {"loss": []}

    if not self.compiled:
      raise NotImplementedError("Model non compilé. Pas de loss")

    for epoch in range(epochs):
      predictions = self.forward(X)
      loss = self.loss.forward(predictions, y)
      loss_derivee = self.loss.backward()
      self.backward(loss_derivee)

      print(f"Epoch {epoch + 1} ............ loss : {loss}")

      #update
      self.step()

    return self.history


  def __repr__(self):
    out = "Layers............................."
    for couche in self.layers:
      out += '\n ' + str(couche)
    return out

In [ ]:
model = Model([Dense(neurones=2, ),
               Dense(neurones=1)])

In [ ]:
model

Layers.............................
 DenseLayer(neurones=2)
 DenseLayer(neurones=1)

In [ ]:
X

array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [ ]:
model.forward(X)

array([[ 2.35895624],
       [ 3.02897647],
       [-0.03274549],
       [ 1.54593604]])

In [ ]:
loss_derivee = np.array([[0.26120692],
       [1.31940416],
       [0.38818031],
       [1.0299873 ]])

In [ ]:
loss_derivee.shape

(4, 1)

In [ ]:
model.backward(loss_derivee)

array([[ 0.06943976,  0.02902934, -0.02192428],
       [ 0.350753  ,  0.14663252, -0.11074354],
       [ 0.10319462,  0.04314058, -0.03258173],
       [ 0.27381385,  0.11446806, -0.08645148]])

In [ ]:
model.layers[0].output

array([[ 4.98401349,  5.52826961],
       [ 7.03866549,  8.06366376],
       [-0.311441  ,  3.80240508],
       [ 3.88104201,  7.44603618]])

In [ ]:
model.layers[0].output.shape

(4, 2)

In [ ]:
model.layers[1].output

array([[ 2.35895624],
       [ 3.02897647],
       [-0.03274549],
       [ 1.54593604]])

In [ ]:
model.layers[1].backward(loss_derivee)

array([[ 0.12974517, -0.03611559],
       [ 0.65536672, -0.18242649],
       [ 0.19281465, -0.05367148],
       [ 0.51160927, -0.14241047]])

In [ ]:
model.layers[1].backward(loss_derivee).shape

(4, 2)

In [ ]:
model.layers[0].output.shape

(4, 2)

In [ ]:
model.layers[0].backward(model.layers[1].backward(loss_derivee))

array([[ 0.06943976,  0.02902934, -0.02192428],
       [ 0.350753  ,  0.14663252, -0.11074354],
       [ 0.10319462,  0.04314058, -0.03258173],
       [ 0.27381385,  0.11446806, -0.08645148]])

In [ ]:
# Calcul de l'erreur
X

array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [ ]:
Y

array([[ 0.97586535],
       [-2.34996566],
       [-0.27417641],
       [-1.16578877]])

In [ ]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])

In [ ]:
model

Layers.............................
 DenseLayer(neurones=2) with Sigmoid
 DenseLayer(neurones=1)

In [ ]:
P = model.forward(X)
P

array([[1.00330788],
       [1.00574651],
       [0.72243492],
       [0.99617929]])

In [ ]:
mse = Loss()
loss = mse.forward(P, Y)
loss

4.232224338855465

In [ ]:
loss_derivee = mse.backward()
loss_derivee

array([[0.01372126],
       [1.67785609],
       [0.49830566],
       [1.08098403]])

In [ ]:
model.layers

[DenseLayer(neurones=2) with Sigmoid, DenseLayer(neurones=1)]

In [ ]:
model.backward(loss_derivee) # c'est ici que tout se passe

In [ ]:
# Maintenant, il faut qu'on recupère les paramètres et les gradients DANS LA CLASSE Dense

#on a déjà le paramètre params
# on a créee dans Dense la fonction get_params_gradient

In [ ]:
model.layers[0].params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [ ]:
model.layers[1].params

[array([[ 0.49671415],
        [-0.1382643 ]]),
 array([[0.64768854]])]

In [320]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
loss_derivee = mse.backward()
model.backward(loss_derivee)


4.232224338855465


In [ ]:
model.layers[0].params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [ ]:
model.layers[0].derivee_param # et si on le fait

AttributeError: ignored

In [ ]:
model.layers[0].get_params_gradient() # et si on l'applique dès le backward

In [ ]:
model.layers[0].derivee_params

[array([[-0.29899998,  0.00704913],
        [ 0.17775935, -0.00376545],
        [ 0.22291882, -0.00467425]]),
 array([[ 0.07181167, -0.0016388 ]])]

In [ ]:
model.layers[1].params

[array([[ 0.49671415],
        [-0.1382643 ]]),
 array([[0.64768854]])]

In [ ]:
model.layers[1].derivee_params

AttributeError: ignored

In [ ]:
model.layers[1].get_params_gradient()

In [ ]:
model.layers[1].derivee_params

[array([[2.95981339],
        [3.25877618]]),
 array([[3.27086705]])]

# Model params et gradients

In [ ]:
# Et si on ajoutait un paramètre params et derivee_params pour le model aussi

# get_params and get_params_gradients ajouté à Model

In [310]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
print("loss : ", loss)
loss_derivee = mse.backward()
model.backward(loss_derivee)

loss :  4.232224338855465


In [ ]:
model.layers[0]

DenseLayer(neurones=2) with Sigmoid

In [ ]:
model.layers[0].params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [316]:
model.layers[0].derivee_params

[array([[-0.2512814 ,  0.00819082],
        [ 0.15224099, -0.00428773],
        [ 0.19010867, -0.00548481]]),
 array([[ 0.06083352, -0.00186526]])]

# Optimizer

In [ ]:
class SGD():

    def __init__(self, learning_rate = 0.01):

        self.learning_rate = learning_rate

    def step(self): # a été ajouté au model

        for (param, derivee_param) in zip(self.parameters(), self.gradients() ):


            param -= self.learning_rate * derivee_param

In [ ]:
# updating Model to add parameters and gradients

In [311]:
model.step()

AttributeError: ignored

In [ ]:
model.layers[0].params

[array([[ 0.49970415, -0.13833479],
        [ 0.64591094,  1.52306751],
        [-0.23638256, -0.23409021]]),
 array([[1.5784947 , 0.76745112]])]

In [312]:
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
print("loss : ", loss)
loss_derivee = mse.backward()
model.backward(loss_derivee)

loss :  4.232224338855465


In [ ]:
model.params

[[array([[ 0.49970415, -0.13833479],
         [ 0.64591094,  1.52306751],
         [-0.23638256, -0.23409021]]),
  array([[1.5784947 , 0.76745112]])],
 [array([[ 0.46711602],
         [-0.17085206]]),
  array([[0.61497987]])]]

In [ ]:
model.layers[0].params

[array([[ 0.49970415, -0.13833479],
        [ 0.64591094,  1.52306751],
        [-0.23638256, -0.23409021]]),
 array([[1.5784947 , 0.76745112]])]

In [ ]:
model.step()

In [ ]:
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
print("loss : ", loss)
loss_derivee = mse.backward()
model.backward(loss_derivee)

loss :  3.67576120170947


In [ ]:
model.step()

In [ ]:
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
print("loss : ", loss)
loss_derivee = mse.backward()
model.backward(loss_derivee)

loss :  3.4416665234297223


In [ ]:
model.parameters()

<generator object Model.parameters at 0x7ff171a27450>

In [ ]:
list(model.parameters())

[array([[ 0.50449769, -0.13850298],
        [ 0.64299117,  1.52315494],
        [-0.24002424, -0.23397722]]),
 array([[1.57733158, 0.76748915]]),
 array([[ 0.41264642],
        [-0.23070506]]),
 array([[0.55491766]])]

# La fonction fit

In [ ]:
# ici on ajoute la fonction compile et fit pour avoir le loss

In [309]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])
P = model.forward(X)
mse = Loss()
loss = mse.forward(P, Y)
print("loss : ", loss)
loss_derivee = mse.backward()
model.backward(loss_derivee)

loss :  4.232224338855465


In [313]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])

In [314]:
mse = Loss()
model.compile(loss=mse)
history = model.fit(X, Y, epochs=10)

TypeError: ignored

In [ ]:
# add learning_rate to compile

In [315]:
model = Model([Dense(neurones=2, activation=Sigmoid()),
               Dense(neurones=1)])
mse = Loss()
model.compile(loss=mse, learning_rate=0.001)
history = model.fit(X, Y, epochs=13)

Epoch 1 ............ loss : 4.232224338855465
Epoch 2 ............ loss : 4.202054010257103
Epoch 3 ............ loss : 4.172218932083374
Epoch 4 ............ loss : 4.142715238119775
Epoch 5 ............ loss : 4.113539110177874
Epoch 6 ............ loss : 4.084686777417401
Epoch 7 ............ loss : 4.056154515679636
Epoch 8 ............ loss : 4.0279386468318705
Epoch 9 ............ loss : 4.0000355381227415
Epoch 10 ............ loss : 3.972441601548228
Epoch 11 ............ loss : 3.9451532932281066
Epoch 12 ............ loss : 3.918167112792662
Epoch 13 ............ loss : 3.8914796027794782


# Comparaison avec Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
model = Sequential([Dense(units=2, activation='sigmoid'),
                   Dense(units=1)])
model.compile(loss='mse', optimizer=SGD(learning_rate=0.1))
history = model.fit(X, Y, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 688ms/step - loss: 1.8038
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 1.7147
Epoch 3/10
1/1 [==============================] - 0s 8ms/step - loss: 1.6647
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 1.6314
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 1.6061
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 1.5851
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 1.5667
Epoch 8/10
1/1 [==============================] - 0s 10ms/step - loss: 1.5499
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: 1.5341
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 1.5193


TypeError: ignored

In [ ]:
panier = ["bananes", "melon", "citron"]
nombres = [1, 2, 3]
# http://dabeaz.com/coroutines/Coroutines.pdf

In [ ]:
def get_fruits():
  panier = ["bananes", "melon", "citron"]

  for fruit in panier:
    yield fruit

In [ ]:
for fruit in get_fruits():
  print(fruit)
  fruit = fruit.upper()

bananes
melon
citron


In [ ]:
list(get_fruits())

['bananes', 'melon', 'citron']

In [ ]:
def get_fruits():
  panier = ["bananes", "melon", "citron"]
  yield from panier

In [ ]:
for fruit in get_fruits():
  print(fruit.upper())
  fruit = fruit.upper()

BANANES
MELON
CITRON


In [ ]:
list(get_fruits())

['bananes', 'melon', 'citron']